In [289]:
from collections import defaultdict
import numpy as np

In [290]:
def read_input(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    return lines


def check_input_format(input_):
    for i in input_:
        if not i.isalnum():
            raise Exception("Node label has to be alphanumeric")
            

def check_missing_leaves(label2val, leaf_nodes, adj_list):
    for leaf in leaf_nodes:
        if leaf not in label2val:
            lis = [li for li in adj_list.values() if leaf in li]
            parent_nodes = [k for li in lis for k in adj_list if adj_list[k]==li]
            raise Exception(f"child node {leaf} of {parent_nodes} not found")

            
def find_root(child2parent, adj_list):
    root_set = set()
    for node in adj_list:
        if node not in child2parent:
            root_set.add(node)
    check_multiple_root(root_set)
    return next(iter(root_set))


def check_multiple_root(root_set):
    if len(root_set) > 1:
        raise Exception(f"multiple roots: {root_set}")
            
            
            
def parse_input(filename):
    lines = read_input(filename=filename)
    lines = [l.strip('\n') for l in lines if l != '\n']  #remove extra spaces from input
        
    adj_list = dict()
    label2val = dict()
    child2parent = defaultdict(list)
    labels = set()
    internal_nodes = set()
    leaf_nodes = set()
    
    for line in lines:
        if ":" in line:
            src, children = line.split(": ")[0], line.split(": ")[-1].lstrip('[').rstrip(']').split(", ")
            check_input_format([src])
            check_input_format(children)
            adj_list[src] = children
            for c in children:
                child2parent[c].append(src)
            labels.add(src)
            internal_nodes.add(src)
            labels.update(children)
            label2val[src] = None

        if "=" in line:
            label_, val = line.split("=")[0], int(line.split("=")[-1])
            check_input_format([label_])
            label2val[label_] = val
    
    leaf_nodes = labels - internal_nodes
    check_missing_leaves(label2val=label2val, leaf_nodes=leaf_nodes, adj_list=adj_list)
    root = find_root(child2parent, adj_list)
            
    return adj_list, label2val, root

In [291]:
adj_list, label2val, root = parse_input("example7.txt")
adj_list, label2val, root

({'a': ['a1', 'a2', 'a3'],
  'a1': ['b', 'c', 'd'],
  'a3': ['x', 'y', 'z'],
  'a2': ['f', 'g', 'h']},
 {'a': None,
  'a1': None,
  'a3': None,
  'a2': None,
  'b': -10,
  'c': 10,
  'd': 5,
  'x': 2,
  'y': -1,
  'z': 8,
  'f': 4,
  'g': -3,
  'h': 7},
 'a')

In [301]:
class Node:
    def __init__(self, label, value=None, node_type='min'):
        self.label = label
        self.value = value
        self.node_type = node_type
        self.children = []
        
    def __repr__(self,):
        return str(self.label)
    
    def __str__(self,):
        return f"({self.name}, {self.value}, {self.node_type})"
    
    
class DAG:
    def __init__(self, adj_list=None, root_label=None, root_node_type='max', n=None, label2val=None, v=None):
        self.adj_list = adj_list if adj_list else defaultdict(list) # {'a': ['a1', 'a2', 'a3'], ...}
        self.label2node = {}  # {'a': Node('a'), 'a1': Node('a1'), 'a2': Node('a2'), ...}
        self.root_label = root_label # 'a'
        self.root_node_type = root_node_type
        self.root = None
        self.n = n
        self.label2val=label2val
        self.v=v
        
    def __repr__(self,):
        return f"DAG({str(self.adj_list)})"
    
    def __str__(self):
        return f"DAG({str(self.adj_list)})"
    
    def create_node_graph(self):
        if self.root_label is None:
            raise ValueError("Root cannot be None before instantiating graph!")
        
        self.root = self._create_nodes(self.root_label, self.root_node_type)
        if not verbose:
            res = max([child.value for child in self.root.children]) if self.root.node_type == 'max' else min([child.value for child in self.root.children])
            choice = self.root.children[np.argmax([child.value for child in self.root.children])].label if self.root.node_type == 'max' else self.root.children[np.argmin([child.value for child in self.root.children])].label
            print(f"({self.root.node_type}){self.root.label} chooses {choice} for {self.root.value}")

        
    def _create_nodes(self, root_label, root_node_type):
        # root_label = 'a'
        
        # Check if root label already has a Node object
        if root_label in self.label2node:
            print(f"root_label = {root_label} already in label2node.")
            root_node = self.label2node[root_label]
        else:
            print(f"root_label = {root_label} not in label2node.")
            root_node = Node(label=root_label, value=self.label2val[root_label], node_type=root_node_type)
            self.label2node[root_label] = root_node
            
        # root_node = Node('a')
#         print(self.label2node)
        
        child_values = []
        if root_label in self.adj_list:
            print(f"Present: {self.adj_list[root_label]}")
            
            for child_label in self.adj_list[root_label]:
                if root_node_type == 'max':
                    child_node_type = 'min'
                if root_node_type == 'min':
                    child_node_type = 'max'
                child_node = self._create_nodes(child_label, child_node_type)
                if self.n and root_node_type == 'max' and child_node.value == self.n:
                    root_node.value = self.n
                    choice = child_node.label
                    if verbose:
                        print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}")
                    return root_node
                if self.n and root_node_type == 'min' and child_node.value == -self.n:
                    root_node.value = -self.n
                    choice = child_node.label
                    if verbose:
                        print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}")
                    return root_node
                child_values.append(child_node.value)
                root_node.children.append(child_node)
            

            root_node.value = max(child_values) if root_node.node_type == 'max' else min(child_values)
            choice = self.adj_list[root_label][np.argmax(child_values)] if root_node.node_type == 'max' else self.adj_list[root_label][np.argmin(child_values)]
            if verbose:
                print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}")
                
        return root_node

In [303]:
adj_list, label2val, root = parse_input("example1.txt")
print(adj_list, label2val, root)
verbose=False
dag = DAG(adj_list=adj_list, root_label=root, root_node_type='min', label2val=label2val, v=verbose)
dag.create_node_graph()

{'a': ['a1', 'a2', 'a3'], 'a1': ['b', 'c'], 'a3': ['xy', 'wx'], 'a2': ['b19', 'b29']} {'a': None, 'a1': None, 'a3': None, 'a2': None, 'b': -4, 'c': 3, 'b19': 5, 'b29': 2, 'xy': -1, 'wx': 8} a
root_label = a not in label2node.
Present: ['a1', 'a2', 'a3']
root_label = a1 not in label2node.
Present: ['b', 'c']
root_label = b not in label2node.
root_label = c not in label2node.
root_label = a2 not in label2node.
Present: ['b19', 'b29']
root_label = b19 not in label2node.
root_label = b29 not in label2node.
root_label = a3 not in label2node.
Present: ['xy', 'wx']
root_label = xy not in label2node.
root_label = wx not in label2node.
(min)a chooses a3 for 3


(max)a1 chooses c for 3
(max)a2 chooses b19 for 5
(max)a3 chooses wx for 8
(min)a chooses a1 for 3


In [288]:
s = str(['minmax.py', '5', 'min', 'example1.txt'])
s.lstrip("[").rstrip(']')[1:-1]

"minmax.py', '5', 'min', 'example1.txt"

In [126]:
def condition(alpha, beta):
    return beta <= alpha

In [315]:
## Alpha Beta Pruning
## ------------------

class Node:
    def __init__(self, label, value=None, node_type='min', alpha=float("-inf"), beta=float("inf")):
        self.label = label
        self.value = value
        self.node_type = node_type
        self.children = []
        self.alpha = alpha
        self.beta = beta
        
    def __repr__(self,):
        return str(self.label)
    
    def __str__(self,):
        return f"({self.label}, {self.value}, {self.node_type}, {self.alpha}, {self.beta})"
    
    
class DAG:
    def __init__(self, adj_list=None, root_label=None, root_node_type='max', n=None):
        self.adj_list = adj_list if adj_list else defaultdict(list) # {'a': ['a1', 'a2', 'a3'], ...}
        self.label2node = {}  # {'a': Node('a'), 'a1': Node('a1'), 'a2': Node('a2'), ...}
        self.root_label = root_label # 'a'
        self.root_node_type = root_node_type
        self.root = None
        self.n = n
        
    def __repr__(self,):
        return f"DAG({str(self.adj_list)})"
    
    def __str__(self):
        return f"DAG({str(self.adj_list)})"
    
    def create_node_graph(self):
        if self.root_label is None:
            raise ValueError("Root cannot be None before instantiating graph!")
        alpha = float('-inf')
        beta = float('inf')
        self.root = self._create_nodes(self.root_label, self.root_node_type, alpha, beta)

        
    def _create_nodes(self, root_label, root_node_type, root_alpha, root_beta):
        # root_label = 'a'
        
        # Check if root label already has a Node object
        if root_label in self.label2node:
#             print(f"root_label = {root_label} already in label2node.")
            root_node = self.label2node[root_label]
        else:
#             print(f"root_label = {root_label} not in label2node.")
            root_node = Node(label=root_label,
                             value=label2val[root_label],
                             node_type=root_node_type,
                             alpha=root_alpha,
                             beta=root_beta)
            self.label2node[root_label] = root_node
            
        child_nodes_visited = []
        local_verbose = True

        if root_label in self.adj_list:
#             print(f"Present: {self.adj_list[root_label]}")
#             print(f"Rootnode: {root_node.label}, alpha:{root_node.alpha}, beta:{root_node.beta}")
            for child_label in self.adj_list[root_label]:
                if condition(root_node.alpha, root_node.beta):
                    print(f"pruning child: {child_node.label} of parent: {root_label}.........")
                    local_verbose = False
                    break;
                if root_node_type == 'max':
                    child_node_type = 'min'
                if root_node_type == 'min':
                    child_node_type = 'max'
                child_node = self._create_nodes(child_label, child_node_type, root_node.alpha, root_node.beta)
                if root_node.node_type == 'max':
                    if child_node.value > root_node.alpha:
                        root_node.alpha = child_node.value
                        root_node.value = root_node.alpha
                        choice = child_node.label
                        if self.n and root_node.value == self.n:
                            if local_verbose and verbose:
                                print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}. Alpha:{root_node.alpha}, Beta:{root_node.beta}")
                            return root_node
                if root_node.node_type == 'min':
                    if child_node.value < root_node.beta:
                        root_node.beta = child_node.value
                        root_node.value = root_node.beta
                        choice = child_node.label
                        if self.n and root_node.value == -self.n:
                            if local_verbose and verbose:
                                print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}. Alpha:{root_node.alpha}, Beta:{root_node.beta}")
                            return root_node

#                 print(f"Root node {root_node.label} after evaluation: {root_node}")
                root_node.children.append(child_node)
                
            if local_verbose and verbose:
                if not condition(root_node.alpha, root_node.beta):
                    print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}. Alpha:{root_node.alpha}, Beta:{root_node.beta}")
        
        return root_node

In [317]:
adj_list, label2val, labels = parse_input("example7.txt")
verbose=True
dag = DAG(adj_list=adj_list, root_label='a', root_node_type='min', n=10)
dag.create_node_graph()

(max)a1 chooses c for 10. Alpha:10, Beta:inf
(max)a2 chooses h for 7. Alpha:7, Beta:10
(min)a chooses a2 for 7. Alpha:-inf, Beta:7


In [313]:
from collections import defaultdict
import numpy as np

def condition(alpha, beta):
    return beta <= alpha

class Node_ab:
    """Class Node represents a node in the graph and supports alpha-beta pruning"""
    def __init__(self, label, value=None, node_type='min', alpha=float("-inf"), beta=float("inf")):
        self.label = label
        self.value = value
        self.node_type = node_type
        self.children = []
        self.alpha = alpha
        self.beta = beta
        
    def __repr__(self,):
        return str(self.label)
    
    def __str__(self,):
        return f"({self.label}, {self.value}, {self.node_type}, {self.alpha}, {self.beta})"


class DAG_ab:
    """Class DAG represent the grpah input and supports alpha-beta pruning with max value cutoff"""
    def __init__(self, adj_list=None, root_label=None, root_node_type='max', n=None, label2val=None, v=None):
        self.adj_list = adj_list if adj_list else defaultdict(list) # {'a': ['a1', 'a2', 'a3'], ...}
        self.label2node = {}  # {'a': Node('a'), 'a1': Node('a1'), 'a2': Node('a2'), ...}
        self.root_label = root_label # 'a'
        self.root_node_type = root_node_type
        self.root = None
        self.n = n
        self.label2val = label2val
        self.verbose = v
        
    def __repr__(self,):
        return f"DAG({str(self.adj_list)})"
    
    def __str__(self):
        return f"DAG({str(self.adj_list)})"
    
    def create_node_graph(self):
        if self.root_label is None:
            raise ValueError("Root cannot be None before instantiating graph!")
        alpha = float('-inf')
        beta = float('inf')
        self.root = self._create_nodes(root_label=self.root_label,
                                       root_node_type = self.root_node_type,
                                       root_alpha=alpha,
                                       root_beta=beta)

        
    def _create_nodes(self, root_label, root_node_type, root_alpha, root_beta):        
        # Check if root label already has a Node object
        if root_label in self.label2node:
            print(f"root_label = {root_label} already in label2node.")
            root_node = self.label2node[root_label]
        else:
            print(f"root_label = {root_label} not in label2node.")
            root_node = Node_ab(label=root_label,
                             value=self.label2val[root_label],
                             node_type=root_node_type,
                             alpha=root_alpha,
                             beta=root_beta)
            self.label2node[root_label] = root_node
            
        local_verbose = True

        if root_label in self.adj_list:
            for child_label in self.adj_list[root_label]:
                if condition(root_node.alpha, root_node.beta):
                    # print(f"pruning child: {child_node.label} of parent: {root_label}.........")
                    local_verbose = False
                    break;
                if root_node_type == 'max':
                    child_node_type = 'min'
                if root_node_type == 'min':
                    child_node_type = 'max'
                child_node = self._create_nodes(root_label=child_label,
                                                root_node_type=child_node_type,
                                                root_alpha=root_node.alpha,
                                                root_beta=root_node.beta)
                print(f"$$$$$$$$$$$$$$$ {child_node}")
                if root_node.node_type == 'max':
                    if child_node.value > root_node.alpha:
                        root_node.alpha = child_node.value
                        root_node.value = root_node.alpha
                        choice = child_node.label
                        if self.n and root_node.value == self.n:
                            if local_verbose and self.verbose:
                                print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}")
                            return root_node
                if root_node.node_type == 'min':
                    if child_node.value < root_node.beta:
                        root_node.beta = child_node.value
                        root_node.value = root_node.beta
                        choice = child_node.label
                        if self.n and root_node.value == -self.n:
                            if local_verbose and self.verbose:
                                print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}")
                            return root_node

                root_node.children.append(child_node)
                
            if not condition(root_node.alpha, root_node.beta):
                if local_verbose and self.verbose:
                    print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}")
                return root_node

In [314]:
adj_list, label2val, root = parse_input("example1.txt")
print(adj_list)
verbose=True
dag_ab = DAG_ab(adj_list=adj_list, root_label=root, root_node_type='min', v=verbose, label2val=label2val)
dag_ab.create_node_graph()

{'a': ['a1', 'a2', 'a3'], 'a1': ['b', 'c'], 'a3': ['xy', 'wx'], 'a2': ['b19', 'b29']}
root_label = a not in label2node.
root_label = a1 not in label2node.
root_label = b not in label2node.
$$$$$$$$$$$$$$$ None


AttributeError: 'NoneType' object has no attribute 'value'

In [ ]:
dag.root.value